In [1]:
import requests # API / webpage request queries 
import pandas as pd # nice data frames, import and export
from bs4 import BeautifulSoup # HTML parser
import re # regular expressions
from datetime import datetime

Pulled data from [Worldometer](https://www.worldometers.info/coronavirus/). 

In [2]:
url = "https://www.worldometers.info/coronavirus/"
webpage = requests.get(url)
html = BeautifulSoup(webpage.content, "html.parser")

Downloaded webpage URL and scrapped the html

In [3]:
def extract_headers(table):
    headers = table.select("thead th")
    for h in headers:
        for x in h.find_all("br"):
            x.replace_with("_")
    headers = [x.get_text().lower() for x in headers]
    clean_headers = [re.sub("[,\\\/\\n].*", "", x) for x in headers]
    clean_headers = [y + "_pop" if re.search("1m", x) else y 
                     for x, y in zip(headers, clean_headers)]
    clean_headers[8] = "total_cases_pop"
    return(clean_headers)

In [4]:
def extract_data(table): 
    headers = extract_headers(table)
    data = [x.get_text() for x in table.select("td")]
    df = pd.DataFrame({x: data[i:len(data):len(headers)] for i,x
                      in enumerate(headers)})
    return(df)

In [5]:
tables = html.select(".tab-content .main_table_countries_div table")
df = [extract_data(x) for x in tables]

In [6]:
date_str = datetime.now().strftime("%d%b%G_%H%M%S");
df[0].to_csv(("today_%s.csv") % date_str, index = False)
df[1].to_csv(("yesterday_%s.csv") % date_str, index = False)